# Guided Capstone Step 4. Pre-Processing and Training Data Development

**The Data Science Method**  


1.   Problem Identification 


2.   Data Wrangling 
  
 
3.   Exploratory Data Analysis   

4.   **Pre-processing and Training Data Development**  
 * Create dummy or indicator features for categorical variables
  * Standardize the magnitude of numeric features
  * Split into testing and training datasets
  * Apply scaler to the testing set
5.   Modeling 
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

**<font color='teal'> Start by loading the necessary packages as we did in step 3 and printing out our current working directory just to confirm we are in the correct project directory. </font>**

In [145]:
import os
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

os.getcwd()

'/Users/LJ/Documents/Springboard/Guided Project/Ski Resort'

**<font color='teal'>  Load the csv file you created in step 3, remember it should be saved inside your data subfolder and print the first five rows.</font>**

In [146]:
df = pd.read_csv("data/step3_output.csv", index_col = 0)
df = df.reset_index(drop=True)
df

,Name,state,summit_elev,vertical_drop,trams,fastEight,fastSixes,fastQuads,quad,triple,...,SkiableTerrain_ac,Snow Making_ac,daysOpenLastYear,yearsOpen,averageSnowfall,AdultWeekday,AdultWeekend,projectedDaysOpen,NightSkiing_ac,clusters
0,Hilltop Ski Area,Alaska,2090,294,0,0,0,0,0,1,...,30.0,30.0,150.000000,36.0,69.0,30.0,34.00000,152.0,30.0,0
1,Sunrise Park Resort,Arizona,11100,1800,0,0,0,1,2,3,...,800.0,80.0,115.000000,49.0,250.0,74.0,78.00000,104.0,80.0,1
2,Yosemite Ski & Snowboard Area,California,7800,600,0,0,0,0,0,1,...,88.0,0.0,110.000000,84.0,300.0,47.0,47.00000,107.0,0.0,1
3,Boreal Mountain Resort,California,7700,500,0,0,0,1,1,3,...,380.0,200.0,150.000000,54.0,400.0,49.0,64.16681,150.0,200.0,1
4,Dodge Ridge,California,8200,1600,0,0,0,0,1,2,...,862.0,0.0,151.578947,69.0,350.0,78.0,78.00000,140.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,Tyrol Basin,Wisconsin,1160,300,0,0,0,0,0,3,...,32.0,32.0,112.000000,61.0,41.0,17.0,48.00000,103.0,32.0,0
164,Whitecap Mountain,Wisconsin,1750,400,0,0,0,0,1,0,...,400.0,300.0,105.000000,57.0,200.0,60.0,60.00000,118.0,0.0,0
165,Hogadon Basin,Wyoming,8000,640,0,0,0,0,0,0,...,92.0,32.0,121.000000,61.0,80.0,48.0,48.00000,95.0,0.0,1
166,Sleeping Giant Ski Resort,Wyoming,7428,810,0,0,0,0,0,1,...,184.0,18.0,61.000000,81.0,310.0,42.0,42.00000,77.0,0.0,1


## Create dummy features for categorical variables

**<font color='teal'> Create dummy variables for `state`. Add the dummies back to the dataframe and remove the original column for `state`. </font>**

Hint: you can see an example of how to execute this in Aiden's article on preprocessing [here](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-pre-processing-and-training-data-development-fd2d75182967). 

In [147]:
dumbf = pd.get_dummies(df, columns=['state'])
dumbf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168 entries, 0 to 167
Data columns (total 60 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Name                  168 non-null    object 
 1   summit_elev           168 non-null    int64  
 2   vertical_drop         168 non-null    int64  
 3   trams                 168 non-null    int64  
 4   fastEight             168 non-null    int64  
 5   fastSixes             168 non-null    int64  
 6   fastQuads             168 non-null    int64  
 7   quad                  168 non-null    int64  
 8   triple                168 non-null    int64  
 9   double                168 non-null    int64  
 10  surface               168 non-null    int64  
 11  total_chairs          168 non-null    int64  
 12  Runs                  168 non-null    float64
 13  TerrainParks          168 non-null    float64
 14  LongestRun_mi         168 non-null    float64
 15  SkiableTerrain_ac     1

## Standardize the magnitude of numeric features

**<font color='teal'> Using sklearn preprocessing standardize the scale of the features of the dataframe except the name of the resort which we done't need in the dataframe for modeling, so it can be droppped here as well. Also, we want to hold out our response variable(s) so we can have their true values available for model performance review. Let's set `AdultWeekend` to the y variable as our response for scaling and modeling. Later we will go back and consider the `AdultWeekday`, `dayOpenLastYear`, and `projectedDaysOpen`. For now leave them in the development dataframe. </font>**

In [148]:
# first we import the preprocessing package from the sklearn library
from sklearn import preprocessing

# Declare an explanatory variable, called X,and assign it the result of dropping 'Name' and 'AdultWeekend' from the df
X = dumbf.drop(['Name','AdultWeekend'], axis=1)

# Declare a response variable, called y, and assign it the AdultWeekend column of the df 
y = dumbf['AdultWeekend']

# Here we use the StandardScaler() method of the preprocessing package, and then call the fit() method with parameter X 
scaler = preprocessing.StandardScaler().fit(X)

# Declare a variable called X_scaled, and assign it the result of calling the transform() method with parameter X 
X_scaled=scaler.transform(X) 

## Split into training and testing datasets

**<font color='teal'> Using sklearn model selection import train_test_split, and create a 75/25 split with the y = `AdultWeekend`. We will start by using the adult weekend ticket price as our response variable for modeling.</font>**

In [149]:
# Import the train_test_split function from the sklearn.model_selection utility.  
from sklearn.model_selection import train_test_split

# Get the 1-dimensional flattened array of our response variable y by calling the ravel() function on y
y = y.ravel()

# Call the train_test_split() function with the first two parameters set to X_scaled and y 
# Declare four variables, X_train, X_test, y_train and y_test separated by commas 
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=1)

Here we start the actual modeling work. First let's fit a multiple linear regression model to predict the `AdultWeekend` price.

# Guided Capstone Step 5. Modeling


This is the fifth step in the Data Science Method. In the previous steps you cleaned and prepared the datasets. Now it's time to get into the most exciting part: modeling! In this exercise, you'll build three different models and compare each model's performance. In the end, you'll choose the best model for demonstrating insights to Big Mountain management.



### **The Data Science Method**  


1.   Problem Identification 

2.   Data Wrangling 
  
3.   Exploratory Data Analysis 
 
4.   Pre-processing and Training Data Development

5.   **Modeling**
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

## Fit Models with a Training Dataset

**<font color='teal'> Using sklearn, fit the model on your training dataset.</font>**

#### Model 1

In [150]:
#all first model set
from sklearn import linear_model
from sklearn.metrics import explained_variance_score,mean_absolute_error
lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)

**<font color='teal'> Predict on the testing dataset and score the model performance with the y_test set and the y-pred values. The explained variance is a measure of the variation explained by the model. This is also known as the R-squared value. </font>**

Hint: you will have to use the `predict()` method here as it's used in this [DSM article](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-modeling-56b4233cad1b) about modeling. 

In [151]:
# Make a variable called y_pred and assign it the result of calling predict() on our model variable with parameter X_test
y_pred = model.predict(X_test)

## Review Model Outcomes — Iterate over additional models as needed

In [152]:
# You might want to use the explained_variance_score() and mean_absolute_error() metrics.
# To do so, you will need to import them from sklearn.metrics. 
# You can plug y_test and y_pred into the functions to evaluate the model

In [153]:
explained_variance_score(y_test, y_pred)

-1.173932850367627e+23

In [154]:
mean_absolute_error(y_test, y_pred)

1042453982166.0212

**<font color='teal'> Print the intercept value from the linear model. </font>**

Hint: our linear regression model `lm` has an attribute `intercept_` for the intercept

In [155]:
lm.intercept_

-260613495483.07068

**<font color='teal'> The intercept is the mean `AdultWeekend` price for all the resorts given the other characteristics. The addition or subtraction of each of the coefficient values in the regression are numeric adjustments applied to the intercept to provide a particular observation's value for the resulting `AdultWeekend` value. Also, because we took the time to scale our x values in the training data, we can compare each of the coeeficients for the features to determine the feature importances. Print the coefficient values from the linear model and sort in descending order to identify the top ten most important features.</font>** 


Hint: make sure to review the absolute value of the coefficients, because the adjustment may be positive or negative, but what we are looking for is the magnitude of impact on our response variable.

<font color='violet'>
Observations:

-widely inaccurate model based on MAE and variance score
-no meaning to the intercept --> no AdultWeekend price can be interpreted
-possibly due to dummy variables?  why did we leave in categorical data?
</font>

In [156]:
# You might want to make a pandas DataFrame displaying the coefficients for each state like so: 
pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient']).sort_values('Coefficient',ascending=False)


,Coefficient
fastEight,2.647784e+13
state_New York,7.089432e+12
state_Michigan,6.242477e+12
state_California,5.849290e+12
state_Pennsylvania,5.637915e+12
state_New Hampshire,5.415166e+12
state_Wisconsin,5.415166e+12
fastSixes,5.042043e+12
state_Minnesota,4.661982e+12
state_Vermont,4.374491e+12


**<font color='teal'>You should see that the top ten important features are different states. However, the state is not something the managers at the Big Mountain Resort can do anything about. Given that we care more about actionable traits associated with ticket pricing, rebuild the model without the state features and compare the results. </font>**

Hint: Try to construct another model using exactly the steps we followed above. 

#### Model 2

In [157]:
X = df.drop(['Name','state','AdultWeekend'], axis=1)
y = df['AdultWeekend']
scaler = preprocessing.StandardScaler().fit(X)
X_scaled = scaler.transform(X)

In [158]:
# Get the 1-dimensional flattened array of our response variable y by calling the ravel() function on y
y = y.ravel()
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=1)

In [159]:
model = lm.fit(X_train,y_train)
y_pred = model.predict(X_test)
explained_variance_score(y_test, y_pred)

0.7920392711342277

In [160]:
mean_absolute_error(y_test, y_pred)

5.124514599832811

In [161]:
lm.intercept_

57.01615019129187

In [162]:
# You might want to make a pandas DataFrame displaying the coefficients for each state like so: 
pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient']).sort_values('Coefficient',ascending=False)


,Coefficient
AdultWeekday,9.766044e+00
daysOpenLastYear,3.425507e+00
clusters,2.815168e+00
vertical_drop,2.740277e+00
projectedDaysOpen,2.556239e+00
summit_elev,2.086630e+00
SkiableTerrain_ac,2.082974e+00
averageSnowfall,1.817726e+00
triple,1.601730e+00
quad,1.220364e+00


**<font color='teal'> When reviewing our new model coefficients, we see `summit_elev` is now in the number two spot. This is also difficult to change from a management prespective and highly correlated with `base_elev` and `vertical_drop`.  This time, rebuild the model without the state features and without the `summit_elev` and without `base_elev`and compare the results. </font>**

**<font color='violet'> Having very different results than what was intended and not particularly sure why.  Possibly due to step 1 or 2 when cleaning and filling data.  Features of great importance: daysOpenLastYear, vertical_drop, projectedDaysOpen, summit_elev, SkiableTerrain_ac, averageSnowfall -- all of which are logical when thinking about admission prices.  Model #2 is still more accurate than Model #1 and suggests $57 as the mean AdultWeekend price for all the resorts given the other characteristics. </font>**

#### Model 3

In [174]:
X = df.drop(['Name','state','AdultWeekend','summit_elev'], axis=1)
y = df['AdultWeekend']
scaler = preprocessing.StandardScaler().fit(X)
X_scaled = scaler.transform(X)
y = y.ravel()
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=1)

In [175]:
model = lm.fit(X_train,y_train)
y_pred = model.predict(X_test)
explained_variance_score(y_test, y_pred)

0.7602573110742274

In [176]:
mean_absolute_error(y_test, y_pred)

5.318595016244335

In [177]:
lm.intercept_

57.049919168220626

In [178]:
# You might want to make a pandas DataFrame displaying the coefficients for each state like so: 
pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient']).sort_values('Coefficient',ascending=False)

,Coefficient
AdultWeekday,9.591927e+00
daysOpenLastYear,3.213718e+00
vertical_drop,2.906139e+00
SkiableTerrain_ac,2.806239e+00
projectedDaysOpen,2.547699e+00
averageSnowfall,2.470211e+00
clusters,2.272370e+00
triple,1.344294e+00
quad,1.069655e+00
total_chairs,9.481334e-01


## Identify the Final Model

**<font color='teal'> Review the model performances in the table below and choose the best model for proving insights to Big Mountain management about what features are driving ski resort lift ticket prices. Type your choice in the final markdown cell — you will discuss this selection more in the next step of the guided casptone. </font>**

| Model | Explained Variance| Mean Absolute Error|Features Dropped|
| --- | --- | --- | --- |
| Model 1. |-1.173932850367627e+23|1042453982166.0212 |-|
| Model 2. |0.7920392711342277|5.124514599832811|'state'|
| Model 3. |0.7602573110742274|5.318595016244335 |'state','summit_elev','base_elev'|

Model Selection: Model 3 for it's lowest variance and elimination of more distracting explanatory variables.